# 04 - Senate voting smart contract

**Description**: This notebook will serve as the basis for the creation of the voting smart contract for the academic senate. The smart contract should work as follow: 
1. The administrator address can issue a proposal to vote (smart contract)
2. Each senator should opt in to the voting initiative and express their votes
3. If the number of "In favor" exceeds the number of "Against", the proposal is approved

**TODO**
1. Consider that the state of the voting should not be disclosed up until the end of the voting period or the deletion of the contract. How? Local storage?
2. Consider to set up a timelock contract when the voting is created such that all the members receive the coin to vote before proceeding.

### 0. Load the required assets

In [72]:
# Add "algo_util.py" to the list of available dependencies of the project
import sys, os, json
from pyteal import *
from pathlib import Path

codepath = (Path( os.curdir ) / "shared").resolve()
sys.path.append(str(codepath))

In [73]:
# import some utilities
from algo_util import wait_for_confirmation, load_credentials
from algo_util import format_state, read_global_state, read_local_state

In [74]:
# algo-sdk utilities
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, AssetOptInTxn
import algosdk.error
import json
import base64
import datetime
import hashlib
import random

In [75]:
# loading credential utility
from algo_util import load_credentials 

# load credentials from storage
credentials = load_credentials(file_name = "credentials_temp")

print(f"{len(credentials['members_credentials'])} members' credentials loaded from storage")
print(f"Loading also USIV account credentials: {credentials['usiv']['public']}")

32 members' credentials loaded from storage
Loading also USIV account credentials: FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI


Initialize the algorand client to interact with the testnet

In [76]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(
    algod_token='', 
    algod_address=credentials['algod_test'], 
    headers=credentials['purestake_token']
)

#### Step 1: Define the Approval Program
This is the program that handles all the interactions, with the exception of the opting out contract

In [77]:
# store the credentials
usiv_public_address = credentials['usiv']['public']
print(f"The only account that can issue the contract is: {usiv_public_address}")

The only account that can issue the contract is: FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI


In [78]:
# init
voting_creation = Seq(
    [
        # assertion on contract creation
        Assert(
            And(
                # the creator must be the association USIV address
                Txn.sender() == Addr(f"{usiv_public_address}"),
            )
        ),

        ###
        # Globals
        ###
        # App.globalPut(Bytes("Creator"), Global.creator_address()), # creator of the voting app (should be the USIV account)
        App.globalPut(Bytes("VoteInfo"), Bytes("For each proposal, vote in favor, against, or abstain")), # info about the voting
        App.globalPut(Bytes("Verdict"), Bytes("")), # winner of the voting
        # init the voting counter
        App.globalPut(Bytes("TotalOptIn"), Int(0)),
        App.globalPut(Bytes("TotalVotes"), Int(0)),
        # init the votes for the candidates
        App.globalPut(Bytes("In favor"), Int(0)), 
        App.globalPut(Bytes("Against"), Int(0)),
        App.globalPut(Bytes("Abstained"), Int(0)),

        Return(Int(1))
    ]
)

In [79]:
# optin
voting_optin = Seq(
    [
        # assertion on contract optin
        Assert(
            And(
                # the contract does not allow to opt-in if 
                # count of total opt-in is greater than the number of members
                App.globalGet(Bytes("TotalOptIn")) <= Int(len(credentials['members_credentials']))
            ),
        ),

        # control variable for double voting
        App.localPut(Int(0), Bytes("HasVoted"), Bytes("No")),
        # control variable for users in the voting procedure
        App.globalPut(Bytes("TotalOptIn"), Add(App.globalGet(Bytes("TotalOptIn")), Int(1)) ),

        Return(Int(1))
    ]
)

In [80]:
def set_winner():
    """
        Set the verdict of the proposal
    """
    in_favor = App.globalGet(Bytes("In favor"))
    against = App.globalGet(Bytes("Against"))
    abstained = App.globalGet(Bytes("Abstained"))

    return Cond(
        [And(in_favor > against), App.globalPut(Bytes("Verdict"), Bytes("Approved"))],
        [And(against > in_favor), App.globalPut(Bytes("Verdict"), Bytes("Rejected"))],
        [And(in_favor == against), App.globalPut(Bytes("Verdict"), Bytes("Tie"))],
        [Int(1), App.globalPut(Bytes("Winner"), Bytes("Tie"))]
    )

In [81]:
# vote of the user
expressed_vote  = Txn.application_args[0] 

# time-lock contract
# the starting block is pointed to the current block
start_block     = algod_client.status()['last-round']    
# the ending block is a function of the number of members that need to vote, 
# each getting 1000 blocks (~ 4.5 * 100 = 4500 seconds = 75 minutes) 
# and therefore the voting period should be around 33 * 75 minutes
end_block       = start_block + len(credentials['members_credentials']) * 1000


# what happens with the voting?
on_vote = Seq(
    [
        # assertion on total number of votes
        Assert(
            # the contract does not allow to vote if
            # count of total votes is greater than the number of members
            App.globalGet(Bytes("TotalVotes")) <= Int(len(credentials['members_credentials']))
        ),

        # (Extension 1) time-locked contract: voting should be allowed within a specific timeframe
        If( Or(Txn.first_valid() < Int(start_block), Txn.last_valid() > Int(end_block)) )
            .Then(
                # if the voting is not within the timeframe, the transaction is rejected
                Return(Int(0))
            ),


        # if already voted, exit the app
        If( 
            App.localGet(Int(0), Bytes("HasVoted")) == Bytes("Yes"), 
            Return(Int(0)) # exit the app
        ),

        # Check if vote is valid
        App.globalPut(expressed_vote, Add(App.globalGet(expressed_vote), Int(1))),
        # the user has already voted
        App.localPut(Txn.sender(), Bytes("HasVoted"), Bytes("Yes") ),
    ]
)

In [82]:
# no_op (voting)
voting_noop = Seq(
    [
        # assertion: check that the total number of votes is less than the total number of opt-in and 38 (members of the senate)

        # let the user vote
        on_vote,

        # increment the vote counter 
        App.globalPut(Bytes("TotalVotes"), Add(App.globalGet(Bytes("TotalVotes")), Int(1)) ),
        
        # (Extension 2) voting winner: retrieve the winner of the contract through voting counting on-chain
        # the voting is over when all the members vote for the candidate
        If( App.globalGet(Bytes("TotalVotes")) == Int(len(credentials['members_credentials'])) )
        # If( App.globalGet(Bytes("TotalVotes")) == Int(4) )
            .Then(
                # if so, the voting is over and the winner must be set
                set_winner()
            ),

        Return(Int(1))
    ]
)

In [83]:
# update
voting_update = Return( Txn.sender() == Global.creator_address() )

In [84]:
# closing the contract
voting_closeout = Return( Int(1) )

In [85]:
# deletion (allowed if the creator and the requestor are equivalent)
voting_delete = Return(Txn.sender() == Global.creator_address() )

Pack up all the possible operations in a condition statement to be approved

In [86]:
voting_approval_pyteal = Cond(
    [Txn.application_id() == Int(0), voting_creation],
    [Txn.on_completion() == OnComplete.OptIn, voting_optin],
    [Txn.on_completion() == OnComplete.NoOp, voting_noop],
    [Txn.on_completion() == OnComplete.CloseOut, voting_closeout],
    [Txn.on_completion() == OnComplete.UpdateApplication, voting_update],
    [Txn.on_completion() == OnComplete.DeleteApplication, voting_delete],
)

And finally, let's build and compile the smart contract into the teal language


In [87]:
voting_approval_pyteal = compileTeal(
    voting_approval_pyteal,
    mode=Mode.Application,
    version=5
)

#### Step 1b: Define the Clear State program
This is the part of the contract that handles forced opt-outs

In [88]:
voting_clear_pyteal = Approve()

In [89]:
voting_clear_pyteal = compileTeal(
    voting_clear_pyteal,
    mode=Mode.Application,
    version=5
)

Compile both pyteal programs into a byte-encoded contract

In [90]:
voting_approval_b64 = algod_client.compile(voting_approval_pyteal)
voting_Approval =  base64.b64decode(voting_approval_b64['result'])

voting_clear_b64 = algod_client.compile(voting_clear_pyteal)
voting_Clear =  base64.b64decode(voting_clear_b64['result'])

#### Step 2: Deploy the smart contract

First, let's retrieve the account that should deploy the smart contract

In [91]:
# store the credentials
usiv_public_address = credentials['usiv']['public']
print(f"The only account that can issue the contract is: {usiv_public_address}")

The only account that can issue the contract is: FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI


As a first step, let's create the application

In [92]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()

# How much space do we need?
global_ints = 5    # for TotalVotes, TotalOptIn, Rossi, Smith, Meier
global_bytes = 2   # for VoteInfo, Winner
voting_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 0     # None
local_bytes = 1    # for HasVoted
voting_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = usiv_public_address,          # <-- sender public
      sp = sp,                               # <-- sp
      on_complete = 0,                       # <- what to do when finished (nothing)
      approval_program = voting_Approval,   # <-- approval program 
      clear_program = voting_Clear,         # <-- clear program 
      global_schema = voting_global_schema, # <-- reserve global space 
      local_schema = voting_local_schema    # <-- reserve local space
    )

Then, let's sign

In [93]:
usiv_private_address = credentials['usiv']['private']

And deploy the smart contract

In [94]:
# Step 2: sign transaction
stxn = txn.sign(usiv_private_address)

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  41168818.
Waiting for round 41168818 to finish.
Waiting for round 41168819 to finish.
Transaction 6VW7IOEUB4BJRJKMQDHXGOQNB5R3EOG63IYMUBCYPADRW2NLBFWQ confirmed in round 41168820.


**Important**: retrieve the app_id to be used later to issue operation against the smart contract

In [95]:
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

Created new app-id: 685799166


Read the state of the smart contact from the ledger

In [96]:
format_state(read_global_state(algod_client, app_id))

{'TotalOptIn': 0,
 'Abstained': 0,
 'Against': 0,
 'In favor': 0,
 'TotalVotes': 0,
 'Verdict': b'',
 'VoteInfo': 'For each proposal, vote in favor, against, or abstain'}

#### Step 3: Opt-in the smart contract from all the members

#### Information for the opt-in

In [97]:
members = credentials['members_credentials']

In [98]:
def member_optin(member, member_name, app_id):
    """
    Opt-in a member to a voting contract
    """
    # Step 1: prepare transaction
    sp = algod_client.suggested_params()
    txn = transaction.ApplicationOptInTxn(member['public'], sp, app_id)

    # Step 2: sign transaction
    stxn = txn.sign(member['private'])

    # Step 3: send
    txid = algod_client.send_transactions([stxn])

    # Step 4: await confirmation
    txinfo = wait_for_confirmation(algod_client, txid)

    # Log
    print(f"The member '{member_name}' is opt-in and has {algod_client.account_info(member['public'])['assets'][1]['amount']} USIV coin")

In [99]:
import itertools

# loop through the member and issue a vote
[ member_optin(members[member], member, app_id) for member in dict(itertools.islice(members.items(), 0, 32)) ];

Current round is  41168820.
Waiting for round 41168820 to finish.
Waiting for round 41168821 to finish.
Transaction TVTV7RFR2TRWNZV3WN2OSM5FFVO3Y2TX354X2K25OM2BVIXSVDRA confirmed in round 41168822.
The member 'Murphy' is opt-in and has 0 USIV coin
Current round is  41168822.
Waiting for round 41168822 to finish.
Waiting for round 41168823 to finish.
Transaction HCN45QB6SP5N2U3MBOVMV55LZ4N3K7T6ZHARJL5B56CMCYLSLC7Q confirmed in round 41168824.
The member 'Shaffer' is opt-in and has 0 USIV coin
Current round is  41168824.
Waiting for round 41168824 to finish.
Waiting for round 41168825 to finish.
Transaction U6LPH5NELM42UJLO4LYRHCEBOHB5I3JWETSCTREWAWMZMWAF6BTQ confirmed in round 41168826.
The member 'Beck' is opt-in and has 1 USIV coin
Current round is  41168826.
Waiting for round 41168826 to finish.
Waiting for round 41168827 to finish.
Transaction SDFVDFGWQO6HROBR6BCX5MBYCHRPSFMTP5VGNBUTFKREL634NCVA confirmed in round 41168828.
The member 'Smith' is opt-in and has 0 USIV coin
Current ro

What is the state of the contract after the opt-in?

In [100]:
format_state(read_global_state(algod_client,app_id))

{'Abstained': 0,
 'Against': 0,
 'In favor': 0,
 'TotalOptIn': 32,
 'Verdict': b'',
 'VoteInfo': 'For each proposal, vote in favor, against, or abstain',
 'TotalVotes': 0}

#### Step 4: Let's vote!


#### Voting needed information

In [101]:
# first, retrieve the asset id for voting
usiv                = credentials['usiv']
members             = credentials['members_credentials']
idx_coin            = algod_client.account_info(usiv['public'])["assets"][1]["asset-id"]

print(f"The asset (coin) id to be allowed to express the vote is: {idx_coin}")


The asset (coin) id to be allowed to express the vote is: 684625427


#### Issue a vote
We would like each member to send a group transaction composed of two transactions:
1. The first one is to transfer the coin to the voting contract
2. The second one is to vote for the proposal

In [102]:
# function definition for the member to vote
def member_vote(association, member, member_name, app_id, idx_coin):
    """
        Given a member and the contract id, issue a voting preference:
            "In favor",
            "Against",
            "Abstained"
    """

    ##########
    # Step 1: prepare the transactions
    sp = algod_client.suggested_params()

    # retrieve the vote as a random choice between the three candidates
    vote = ["In favor", "Against", "Abstained"][random.randrange(0,3)]

    # 1st transaction -> Asset transfer to make sure the user can vote
    amt_1 = int(1)
    txn_1 = transaction.AssetTransferTxn(
        sender=member['public'], 
        sp=sp, 
        receiver=association['public'], 
        amt=amt_1,
        index=idx_coin)

    # 2nd transaction -> Vote
    txn_2 = transaction.ApplicationNoOpTxn(
        sender=member['public'], 
        sp=sp, 
        index=app_id, 
        app_args=[vote]
    )

    ##########
    # Step 2: Create group transaction by assigning the group_id to the transaction group
    gid = transaction.calculate_group_id([txn_1, txn_2])
    txn_1.group = gid
    txn_2.group = gid

    ##########
    # Step 3: sign each transaction individually
    stxn_1 = txn_1.sign(member['private'])
    stxn_2 = txn_2.sign(member['private'])

    ##########
    # Step 4: send
    txid = algod_client.send_transactions([stxn_1, stxn_2])

    ##########
    # Step 5: wait for condfirmation
    txinfo = wait_for_confirmation(algod_client, txid)

    # Print the state 
    print(f"The member '{member_name}' has expressed the vote: {vote}")

In [103]:
import itertools

# loop through the member and issue a vote
[ member_vote(usiv, members[member], member, app_id, idx_coin) for member in dict(itertools.islice(members.items(), 0, 32)) ];

Current round is  41168942.
Waiting for round 41168942 to finish.
Waiting for round 41168943 to finish.
Transaction K55L6NNM2MCEPCJ5MQTU64OPE5Z3MLDBTOXHKDU77NBBDQAIUSZA confirmed in round 41168944.
The member 'Murphy' has expressed the vote: Abstained
Current round is  41168944.
Waiting for round 41168944 to finish.
Waiting for round 41168945 to finish.
Transaction AR2PXG2ZBU3MTRJNUG2RDNO6ECHIBDH3B5CC3MNKUUMBREFHM3MA confirmed in round 41168946.
The member 'Shaffer' has expressed the vote: Against
Current round is  41168946.
Waiting for round 41168946 to finish.
Waiting for round 41168947 to finish.
Transaction F5UTH5CED46LG2MX466CFKAWM3UT63LUZQFICXKVQBHELLEGNJZQ confirmed in round 41168948.
The member 'Beck' has expressed the vote: In favor
Current round is  41168948.
Waiting for round 41168948 to finish.
Waiting for round 41168949 to finish.
Transaction 3OZY5O3YZR3VN2ZZQ46FLGBW5UMHLH5LOLSUISTKIZ6AVSKCS4LQ confirmed in round 41168950.
The member 'Smith' has expressed the vote: Against

#### Proposal verdict
Let's count the number of "In favor" and "Against" to see the verdict

In [104]:
format_state(read_global_state(algod_client,app_id))

{'Abstained': 10,
 'Against': 13,
 'TotalOptIn': 32,
 'Verdict': b'KJSWUZLDORSWI===',
 'VoteInfo': 'For each proposal, vote in favor, against, or abstain',
 'In favor': 9,
 'TotalVotes': 32}

### Let a member vote again
This should give a failure of approval from the smart contract has the member has already voted. Also the account does not have enough ASA to vote.

In [105]:
import itertools

# loop through the member and issue a vote
[ member_vote(usiv, members[member], member, app_id, idx_coin) for member in dict(itertools.islice(members.items(), 0, 1)) ];

AlgodHTTPError: TransactionPool.Remember: transaction SZBBOWKZAJ7WCIYIXC4XY5SML22IFFS2UWSP5RG7Q65TDSTTON5A: underflow on subtracting 1 from sender amount 0

#### Step 5: Leave the contract after voting

In [106]:
def member_optout(member, member_name, app_id):
    """
        Optout from the contract after voting
    """

    # Step 1: prepare transaction
    sp = algod_client.suggested_params()
    txn = transaction.ApplicationCloseOutTxn(member['public'], sp, app_id)

    # Step 2: sign
    stxn = txn.sign(member['private'])

    # Step 3: send
    txid = algod_client.send_transactions([stxn])

    # Step 4: wait for condfirmation
    txinfo = wait_for_confirmation(algod_client, txid)

    # Log
    print(f"The member '{member_name}' has opt-out from the contract: {app_id}")

In [107]:
import itertools

# loop through the member and issue a vote
[ member_optout(members[member], member, app_id) for member in dict(itertools.islice(members.items(), 0, 32)) ];

Current round is  41169031.
Waiting for round 41169031 to finish.
Waiting for round 41169032 to finish.
Transaction ZO45GSPLVNE5E6UT74MXBWHTRAXTCLWEX5Y3ULPZD4VQ7F6KRUJA confirmed in round 41169033.
The member 'Murphy' has opt-out from the contract: 685799166
Current round is  41169033.
Waiting for round 41169033 to finish.
Waiting for round 41169034 to finish.
Transaction SYENY7NTRARLTSMMAOGETENTONJL4UTOJ7Z4VU4PYIHVLFKVXOLA confirmed in round 41169035.
The member 'Shaffer' has opt-out from the contract: 685799166
Current round is  41169035.
Waiting for round 41169035 to finish.
Waiting for round 41169036 to finish.
Transaction I4SMUGBVD5E2YXMQRZ3WZQ5SXA7DITKFFG5IH6PXFAR2Z5WUFKUA confirmed in round 41169037.
The member 'Beck' has opt-out from the contract: 685799166
Current round is  41169037.
Waiting for round 41169037 to finish.
Waiting for round 41169038 to finish.
Transaction GXDIDLZKG4G6MWYUDQPGCQQYBL4KHIVJVMZ2W6SGOU5ZJF5TGJQA confirmed in round 41169039.
The member 'Smith' has opt

#### Check again the senate account for balance

In [108]:
print(f'The academic senate account has {algod_client.account_info(usiv["public"])["assets"][1]["amount"]} USIV coins') 

The academic senate account has 1000 USIV coins
